In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('kospi200_raw.csv')
data

,Unnamed: 0,Code,Name,Close,ChangeCode,Changes,ChagesRatio,Amount,Marcap,Stocks,BPS,PER,PBR,EPS,DIV,DPS
0,0,5930,삼성전자,75200,2,-2400,-3.09,2312008038097,448927647760000,5969782550,52002,35.29,1.45,2131,1.92,1444
1,1,660,SK하이닉스,202500,3,0,0.00,625798561849,147420478912500,728002365,77752,0.00,2.60,0,0.59,1200
2,2,373220,LG에너지솔루션,342000,2,-18000,-5.00,134975298500,80028000000000,234000000,86328,64.69,3.96,5287,0.00,0
3,3,5380,현대차,261000,2,-4000,-1.51,251331128000,54657625851000,209416191,351861,5.71,0.74,45703,4.37,11400
4,4,207940,삼성바이오로직스,727000,2,-18000,-2.42,65161519000,51743498000000,71174000,138119,60.33,5.26,12051,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,32350,롯데관광개발,9730,2,-260,-2.60,5431840560,741288505370,76185869,913,0.00,10.66,0,0.00,0
196,196,670,영풍,375000,1,5000,1.35,963490000,690765000000,1842040,2216785,0.00,0.17,0,2.67,10000
197,197,16380,KG스틸,6890,1,120,1.77,3672653580,689061300330,100008897,18893,2.94,0.36,2342,2.90,200
198,198,5250,녹십자홀딩스,14240,2,-220,-1.52,608767590,669681710400,47028210,21591,0.00,0.66,0,2.11,300


In [3]:
data.columns

Index(['Unnamed: 0', 'Code', 'Name', 'Close', 'ChangeCode', 'Changes',
       'ChagesRatio', 'Amount', 'Marcap', 'Stocks', 'BPS', 'PER', 'PBR', 'EPS',
       'DIV', 'DPS'],
      dtype='object')

In [4]:
def convert_percent_to_float(df, column_list):
    for column in column_list:
        # Check if any value in the column contains a '%'
        if df[column].dtype == object and df[column].str.contains('%').any():
            # Remove '%' sign and convert to float
            df[column] = df[column].str.replace('%', '').astype(float) / 100
            print(column)

# Apply the conversion function
convert_percent_to_float(data, data.columns)

In [5]:
numeric_cols = data.select_dtypes(include=[np.number]).columns
print(numeric_cols)

log_transform_cols = data[numeric_cols].columns[(data[numeric_cols] > 1e+5).any()]
print(log_transform_cols)

Index(['Unnamed: 0', 'Code', 'Close', 'ChangeCode', 'Changes', 'ChagesRatio',
       'Amount', 'Marcap', 'Stocks', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'],
      dtype='object')
Index(['Code', 'Close', 'Amount', 'Marcap', 'Stocks', 'BPS'], dtype='object')


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np

def signed_log1p(x):
    return np.sign(x) * np.log1p(np.abs(x))

data[log_transform_cols] = data[log_transform_cols].apply(signed_log1p)

In [7]:
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
data

,Unnamed: 0,Code,Name,Close,ChangeCode,Changes,ChagesRatio,Amount,Marcap,Stocks,BPS,PER,PBR,EPS,DIV,DPS
0,-1.723412,-0.694576,삼성전자,0.295913,0.505423,-0.319896,-0.942141,3.510527,4.225543,3.655631,0.033643,-0.076959,-0.132771,-0.477645,-0.172775,-0.110235
1,-1.706091,-1.840365,SK하이닉스,1.156274,2.568376,0.308031,0.404117,2.632473,3.251776,1.943052,0.343367,-0.101993,0.270255,-0.712923,-0.769713,-0.208808
2,-1.688771,1.468327,LG에너지솔루션,1.611450,0.505423,-4.401424,-1.774294,1.601837,2.717563,1.019288,0.423931,-0.056103,0.746878,-0.129200,-1.034519,-0.693590
3,-1.671450,-0.745395,현대차,1.376692,0.505423,-0.738514,-0.253763,2.019537,2.384145,0.928947,1.505845,-0.097942,-0.381596,4.333021,0.926846,3.911839
4,-1.654129,1.162889,삼성바이오로직스,2.266429,0.505423,-4.401424,-0.650234,1.112552,2.336234,0.050588,0.785801,-0.059196,1.202472,0.617595,-1.034519,-0.693590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.654129,0.191296,롯데관광개발,-1.480122,0.505423,0.240006,-0.728657,-0.556818,-1.376396,0.105973,-3.078121,-0.101993,3.094943,-0.712923,-1.034519,-0.693590
196,1.671450,-1.832524,영풍,1.691455,-1.557529,1.616213,0.992288,-1.718832,-1.438124,-2.923612,2.923097,-0.101993,-0.581356,-0.712923,0.163844,3.346260
197,1.688771,-0.164067,KG스틸,-1.779856,-1.557529,0.339427,1.175274,-0.819771,-1.440283,0.327422,-0.745954,-0.099907,-0.514770,-0.454349,0.267073,-0.612793
198,1.706091,-0.758165,녹십자홀딩스,-1.149374,0.505423,0.250471,-0.258120,-2.027313,-1.465229,-0.286676,-0.643175,-0.101993,-0.409632,-0.712923,-0.087498,-0.572394


In [8]:
data = data.drop(['Unnamed: 0', 'Code'], axis=1)
data

,Name,Close,ChangeCode,Changes,ChagesRatio,Amount,Marcap,Stocks,BPS,PER,PBR,EPS,DIV,DPS
0,삼성전자,0.295913,0.505423,-0.319896,-0.942141,3.510527,4.225543,3.655631,0.033643,-0.076959,-0.132771,-0.477645,-0.172775,-0.110235
1,SK하이닉스,1.156274,2.568376,0.308031,0.404117,2.632473,3.251776,1.943052,0.343367,-0.101993,0.270255,-0.712923,-0.769713,-0.208808
2,LG에너지솔루션,1.611450,0.505423,-4.401424,-1.774294,1.601837,2.717563,1.019288,0.423931,-0.056103,0.746878,-0.129200,-1.034519,-0.693590
3,현대차,1.376692,0.505423,-0.738514,-0.253763,2.019537,2.384145,0.928947,1.505845,-0.097942,-0.381596,4.333021,0.926846,3.911839
4,삼성바이오로직스,2.266429,0.505423,-4.401424,-0.650234,1.112552,2.336234,0.050588,0.785801,-0.059196,1.202472,0.617595,-1.034519,-0.693590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,롯데관광개발,-1.480122,0.505423,0.240006,-0.728657,-0.556818,-1.376396,0.105973,-3.078121,-0.101993,3.094943,-0.712923,-1.034519,-0.693590
196,영풍,1.691455,-1.557529,1.616213,0.992288,-1.718832,-1.438124,-2.923612,2.923097,-0.101993,-0.581356,-0.712923,0.163844,3.346260
197,KG스틸,-1.779856,-1.557529,0.339427,1.175274,-0.819771,-1.440283,0.327422,-0.745954,-0.099907,-0.514770,-0.454349,0.267073,-0.612793
198,녹십자홀딩스,-1.149374,0.505423,0.250471,-0.258120,-2.027313,-1.465229,-0.286676,-0.643175,-0.101993,-0.409632,-0.712923,-0.087498,-0.572394


In [9]:
data.to_csv('kospi200_preprocess.csv')